In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="#MySQL/rivage67",
    database="mydb"
)

mycursor = mydb.cursor(buffered=True)

In [ ]:
from datetime import datetime, timedelta
import time


def check_For_State_One(index):
    now = datetime.today()
    print(now)
    mycursor.execute("UPDATE orders SET order_date_time = '{}' WHERE idOrder = '{}' ".format(now,index))
    mycursor.execute("UPDATE orders SET status = '{}' WHERE idOrder = '{}' ".format(1,index))

def check_For_State_Two():
    mycursor.execute("SELECT order_date_time FROM orders WHERE status=1")
    list_time = mycursor.fetchall()
    
    #print(list_time)
    
    for x in list_time:
        if (x[0] + timedelta(minutes=10)) < datetime.today():
            print("send order number to waiting")
            mycursor.execute("UPDATE orders SET status = '{}' WHERE order_date_time = '{}' ".format(2,x[0]))

def check_For_Deliverying():
    mycursor.execute("SELECT * FROM orders ORDER BY order_date_time")
    mycursor.execute("SELECT idCustomer FROM orders WHERE status=2")
    list_time = mycursor.fetchall()
    if len(list_time) == 0:
        return 0;
    get_person = []
    for x in list_time:
        get_person.append(x[0])
    #print(get_person[0])
        
    mycursor.execute("SELECT zipcode FROM customer WHERE idCustomer='{}'".format(get_person[0]))
    zipcode = mycursor.fetchall()
    final_zipcode = []
    for x in zipcode:
        final_zipcode.append(x[0])
    #print(final_zipcode[0])
    
    mycursor.execute("SELECT idOrder FROM orders WHERE idCustomer='{}'".format(get_person[0]))
    idOrder = mycursor.fetchall()
    final_idOrder = []
    for x in idOrder:
        final_idOrder.append(x[0])
    #print(final_idOrder[0])
    
    mycursor.execute("SELECT idOrder FROM delivery_ops WHERE zipcode='{}'".format(final_zipcode[0]))
    result = mycursor.fetchall()
    final_result = []
    for x in result:
        final_result.append(x[0])
    if final_result[0] == None:
        #print("your command is on your way")
        mycursor.execute("UPDATE delivery_ops SET idOrder = '{}' WHERE zipcode='{}' ".format(final_idOrder[0],final_zipcode[0]))
        mycursor.execute("UPDATE orders SET status = '{}' WHERE idOrder='{}' ".format(3,final_idOrder[0]))
    else:
        print("")
        #print("you need to wait at least 30 mins")
        
def check_if_delivered():
    mycursor.execute("SELECT order_date_time FROM orders WHERE status=3")
    list_time = mycursor.fetchall()
    if len(list_time) == 0:
        return 0
    
    #print(list_time)
    for x in list_time:
        if (x[0] + timedelta(minutes=30)) < datetime.today():
            print("your order is delivered")
            mycursor.execute("UPDATE orders SET status = '{}' WHERE order_date_time = '{}' ".format(4,x[0]))
        else:
            print("the order is on the way")
        
        
    
mycursor.execute("UPDATE delivery_ops SET idOrder = NULL WHERE idDriver = 1 ")
mycursor.execute("UPDATE orders SET status = '{}' WHERE idOrder = '{}' ".format(0,1))
mycursor.execute("UPDATE orders SET status = '{}' WHERE idOrder = '{}' ".format(0,2))
mycursor.execute("SELECT * FROM orders WHERE status=0")
size = mycursor.fetchall()
print(size)

for i in size:
    mycursor.execute("SELECT idOrder FROM orders WHERE status = 0 LIMIT 1")
    index = mycursor.fetchall()
    #print("this is the index",index)
    final_index = []
    for x in index:
        final_index.append(x[0])

    mycursor.execute("SELECT idCustomer FROM orders WHERE status = 0 LIMIT 1")
    person = mycursor.fetchall()
    final_person = []
    for x in person:
        final_person.append(x[0])

    #print(final_index)
    #print(final_person)

    mycursor.execute("SELECT zipcode FROM customer WHERE idCustomer = '{}'".format(final_person[0]))
    zipcode = mycursor.fetchall()
    #print(zipcode)

    check_For_State_One(final_index[0])
    time.sleep(2)


while True:
    check_For_State_Two()
    
    check_For_Deliverying()
    
    check_if_delivered()
    
    time.sleep(30)
    
            
            
#statement select order number, zipcode & status (make status)
#ordertime + 10 mins 
#nothing, preparing, waiting, deliverying, delivered

#wait time is the status is at 0 that means you need at least to wait 10 mins + 30 mins (+30 if another pizza is sent)
#wait time is the status is at 1 that means you need at least to wait +-10 mins + 30 mins (+30 if another pizza is sent)
#wait time is the status is at 2 that means you need at least to wait 30 mins (+30 if another pizza is sent)
#wait time is the status is at 3 that means you need at least to wait +-30 mins 
#wait time is the status is at 4 that means you need at least to wait 0 min (pizza is delivered)